<a href="https://colab.research.google.com/github/Chloeyz13/Topic-Modeling-of-Clothing-Reviews/blob/main/LDA-Topic-Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up Pyspark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -qN https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

In [ ]:
!pip install -q findspark

## Install nltk, download nltk stopwords

In [ ]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# Additional words we would like to ignore, for example, for a corpus of emails
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
print(len(stop_words), stop_words)

179 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

## Stemming and lemmatization

In [ ]:
# installing other dependencies
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)
import numpy as np

import re
import string
import pprint

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Import women's e-commerce clothing reviews

In [ ]:
# nltk.download('inaugural')
# nltk.download('punkt')

# from nltk.corpus import inaugural

In [ ]:
# speechesDict = {fileid: inaugural.sents(fileid) for fileid in nltk.corpus.inaugural.fileids()}
# pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(speechesDict)

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/singhj/big-data-repo/main/text-proc/reviews.csv'
df = pd.read_csv(url)
df.head()

,recNo,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df.shape

(23486, 11)

In [ ]:
# paragraph = df.loc[0]['Review Text']
# paragraph

In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# from nltk.tokenize import sent_tokenize, word_tokenize
# sent_text = nltk.sent_tokenize(paragraph)
# sent_text

In [ ]:
# all_tagged = [nltk.word_tokenize(sent) for sent in sent_text]
# all_tagged

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
review_dic = {}
for i in df.index:
  paragraph = str(df.loc[i]['Review Text'])
  sent_text = nltk.sent_tokenize(paragraph)
  all_tagged = [nltk.word_tokenize(sent) for sent in sent_text]
  review_dic[i] = all_tagged

In [ ]:
# pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(review_dic)

In [ ]:
# a = ["who","are", "you"]
# a.extend("?")
# a

In [ ]:
paragraph = df.loc[10]['Review Text']
paragraph

'Dress runs small esp where the zipper area runs. i ordered the sp which typically fits me and it was very tight! the material on the top looks and feels very cheap that even just pulling on it will cause it to rip the fabric. pretty disappointed as it was going to be my christmas dress this year! needless to say it will be going back.'

In [ ]:
# def sent_to_words(sentences):
#     for sentence in sentences:
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# def collect (sentences):
#     speech = []
#     for sent in sentences:
#         for fragment in sent:
#             speech.extend(fragment)
#     return speech

In [ ]:
#my function
def text_to_words(paragraph):
  sent_text = nltk.sent_tokenize(str(paragraph))
  all_tagged = [nltk.word_tokenize(sent) for sent in sent_text]
  flat_list = [item for sublist in all_tagged for item in sublist]
  return flat_list

In [ ]:
text_to_words(paragraph)

['Dress',
 'runs',
 'small',
 'esp',
 'where',
 'the',
 'zipper',
 'area',
 'runs',
 '.',
 'i',
 'ordered',
 'the',
 'sp',
 'which',
 'typically',
 'fits',
 'me',
 'and',
 'it',
 'was',
 'very',
 'tight',
 '!',
 'the',
 'material',
 'on',
 'the',
 'top',
 'looks',
 'and',
 'feels',
 'very',
 'cheap',
 'that',
 'even',
 'just',
 'pulling',
 'on',
 'it',
 'will',
 'cause',
 'it',
 'to',
 'rip',
 'the',
 'fabric',
 '.',
 'pretty',
 'disappointed',
 'as',
 'it',
 'was',
 'going',
 'to',
 'be',
 'my',
 'christmas',
 'dress',
 'this',
 'year',
 '!',
 'needless',
 'to',
 'say',
 'it',
 'will',
 'be',
 'going',
 'back',
 '.']

In [ ]:
df['content'] = df.apply(lambda row: text_to_words(row['Review Text']), axis=1)
df['contentLen'] = df.apply(lambda row: len(text_to_words(row['Review Text'])), axis=1)
df[['recNo', 'content', 'contentLen']]

,recNo,content,contentLen
0,0,"[Absolutely, wonderful, -, silky, and, sexy, a...",8
1,1,"[Love, this, dress, !, it, 's, sooo, pretty, ....",75
2,2,"[I, had, such, high, hopes, for, this, dress, ...",110
3,3,"[I, love, ,, love, ,, love, this, jumpsuit, .,...",31
4,4,"[This, shirt, is, very, flattering, to, all, d...",41
...,...,...,...
23481,23481,"[I, was, very, happy, to, snag, this, dress, a...",31
23482,23482,"[It, reminds, me, of, maternity, clothes, ., s...",48
23483,23483,"[This, fit, well, ,, but, the, top, was, very,...",50
23484,23484,"[I, bought, this, dress, for, a, wedding, i, h...",97


In [ ]:
# combine 'content' into one list
data = []
for i in range(23486):
  data.extend(df.loc[i]['content'])

In [ ]:
len(data) == df['contentLen'].sum()

True

# Install Gensim and spaCy model

In [ ]:
!pip install gensim
!python3 -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-22 01:22:34.704899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 01:22:36.157061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Gensim

import gensim
from gensim import corpora

from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
spacy.load('en_core_web_sm')

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Clean data

In [ ]:
#remove emails
data = [re.sub('\S*@\S*\s?', '', word) for word in data]

# Remove new line characters
data = [re.sub('\s+', ' ', word) for word in data]

# Remove distracting single quotes
data = [re.sub("\'"," ", word) for word in data]

#remove puctuations from data
data = [word for word in data if word not in string.punctuation]

#remove empty strings
data = list(filter(lambda x: x.strip(), data))
data

['Absolutely',
 'wonderful',
 'silky',
 'and',
 'sexy',
 'and',
 'comfortable',
 'Love',
 'this',
 'dress',
 'it',
 ' s',
 'sooo',
 'pretty',
 'i',
 'happened',
 'to',
 'find',
 'it',
 'in',
 'a',
 'store',
 'and',
 'i',
 ' m',
 'glad',
 'i',
 'did',
 'bc',
 'i',
 'never',
 'would',
 'have',
 'ordered',
 'it',
 'online',
 'bc',
 'it',
 ' s',
 'petite',
 'i',
 'bought',
 'a',
 'petite',
 'and',
 'am',
 '5',
 '8',
 'i',
 'love',
 'the',
 'length',
 'on',
 'me-',
 'hits',
 'just',
 'a',
 'little',
 'below',
 'the',
 'knee',
 'would',
 'definitely',
 'be',
 'a',
 'true',
 'midi',
 'on',
 'someone',
 'who',
 'is',
 'truly',
 'petite',
 'I',
 'had',
 'such',
 'high',
 'hopes',
 'for',
 'this',
 'dress',
 'and',
 'really',
 'wanted',
 'it',
 'to',
 'work',
 'for',
 'me',
 'i',
 'initially',
 'ordered',
 'the',
 'petite',
 'small',
 'my',
 'usual',
 'size',
 'but',
 'i',
 'found',
 'this',
 'to',
 'be',
 'outrageously',
 'small',
 'so',
 'small',
 'in',
 'fact',
 'that',
 'i',
 'could',
 'not'

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [ ]:
data_words = list(sent_to_words(data))
data_words = [dw for dw in data_words if len(dw)>0]

In [ ]:
data_words

[['absolutely'],
 ['wonderful'],
 ['silky'],
 ['and'],
 ['sexy'],
 ['and'],
 ['comfortable'],
 ['love'],
 ['this'],
 ['dress'],
 ['it'],
 ['sooo'],
 ['pretty'],
 ['happened'],
 ['to'],
 ['find'],
 ['it'],
 ['in'],
 ['store'],
 ['and'],
 ['glad'],
 ['did'],
 ['bc'],
 ['never'],
 ['would'],
 ['have'],
 ['ordered'],
 ['it'],
 ['online'],
 ['bc'],
 ['it'],
 ['petite'],
 ['bought'],
 ['petite'],
 ['and'],
 ['am'],
 ['love'],
 ['the'],
 ['length'],
 ['on'],
 ['me'],
 ['hits'],
 ['just'],
 ['little'],
 ['below'],
 ['the'],
 ['knee'],
 ['would'],
 ['definitely'],
 ['be'],
 ['true'],
 ['midi'],
 ['on'],
 ['someone'],
 ['who'],
 ['is'],
 ['truly'],
 ['petite'],
 ['had'],
 ['such'],
 ['high'],
 ['hopes'],
 ['for'],
 ['this'],
 ['dress'],
 ['and'],
 ['really'],
 ['wanted'],
 ['it'],
 ['to'],
 ['work'],
 ['for'],
 ['me'],
 ['initially'],
 ['ordered'],
 ['the'],
 ['petite'],
 ['small'],
 ['my'],
 ['usual'],
 ['size'],
 ['but'],
 ['found'],
 ['this'],
 ['to'],
 ['be'],
 ['outrageously'],
 ['small'],


## Create Bigram and Trigram Models

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['absolutely']


## Remove Stopwords, Make Bigrams and Lemmatize

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'LimitOutput': 1000000000})

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print(data_words_nostops)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print(data_words_bigrams)

# In the end, we didn't create trigrams. Should have taken the extra time.

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['absolutely']]


# Topic Modeling

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# Human readable format of corpus (term-frequency)
# There is nothing magical about 2500:2510, I just wanted to examine a random location
print ([[(id2word[id], freq) for id, freq in cp] for cp in corpus[2500:2510]])

[[], [('comfortable', 1)], [('material', 1)], [], [('good', 1)], [('cut', 1)], [], [], [('sleeve', 1)], [('flatter', 1)]]


In [ ]:
# Build LDA model
num_topics = 10
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
doc_lda = lda_model[corpus]

In [ ]:
!pip install pyLDAvis==3.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136189 sha256=8b298b0ac0db743cd3e75425af8134ce6ab0c26b7fb18d09d56599f12934121e
  Stored in directory: /root/.cache/pip/wheels/d2/cc/56/70e7bd4fdf983a26b148dffa850bfb42707601f27d915f5485
Successfully built pyLDAvis


In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [ ]:
print ([itm for itm in dir(doc_lda) if not itm.startswith('__')])
print ([itm for itm in dir(doc_lda.obj) if (not itm.startswith('__')) and (not itm.startswith('_'))])
doc_lda.obj.print_topics(num_topics=-1)

['_adapt_by_suffix', '_load_specials', '_save_specials', '_smart_save', 'add_lifecycle_event', 'chunksize', 'corpus', 'load', 'metadata', 'obj', 'save', 'save_corpus']
['add_lifecycle_event', 'alpha', 'bound', 'callbacks', 'chunksize', 'clear', 'decay', 'diff', 'dispatcher', 'distributed', 'do_estep', 'do_mstep', 'dtype', 'eta', 'eval_every', 'expElogbeta', 'gamma_threshold', 'get_document_topics', 'get_term_topics', 'get_topic_terms', 'get_topics', 'id2word', 'inference', 'init_dir_prior', 'iterations', 'lifecycle_events', 'load', 'log_perplexity', 'minimum_phi_value', 'minimum_probability', 'num_terms', 'num_topics', 'num_updates', 'numworkers', 'offset', 'optimize_alpha', 'optimize_eta', 'passes', 'per_word_topics', 'print_topic', 'print_topics', 'random_state', 'save', 'show_topic', 'show_topics', 'state', 'sync_state', 'top_topics', 'update', 'update_alpha', 'update_eta', 'update_every']


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.444*"color" + 0.181*"skirt" + 0.154*"cut" + 0.107*"high" + 0.044*"end" + 0.014*"week" + 0.000*"wise" + 0.000*"pattern" + 0.000*"rise" + 0.000*"waiste"'),
 (1,
  '0.333*"order" + 0.320*"buy" + 0.156*"pretty" + 0.124*"usually" + 0.000*"adn" + 0.000*"hunt" + 0.000*"sheath" + 0.000*"deliver" + 0.000*"defiantly" + 0.000*"nightgown"'),
 (2,
  '0.329*"look" + 0.115*"material" + 0.112*"sleeve" + 0.110*"petite" + 0.097*"short" + 0.096*"find" + 0.054*"online" + 0.049*"thin" + 0.008*"waiste" + 0.000*"adn"'),
 (3,
  '0.465*"size" + 0.399*"love" + 0.103*"shirt" + 0.000*"adn" + 0.000*"deliver" + 0.000*"platinum" + 0.000*"crosswrap" + 0.000*"slinky" + 0.000*"nightgown" + 0.000*"defiantly"'),
 (4,
  '0.046*"dress" + 0.017*"great" + 0.014*"get" + 0.013*"try" + 0.012*"feel" + 0.012*"flatter" + 0.012*"perfect" + 0.010*"think" + 0.008*"see" + 0.008*"waist"'),
 (5,
  '0.408*"top" + 0.264*"small" + 0.140*"large" + 0.129*"cute" + 0.020*"extra" + 0.000*"bulkier" + 0.000*"me" + 0.000*"red" + 0.000*"c

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.456808 -0.014061       1        1  67.231161
2      0.064462 -0.446003       2        1   4.681650
9      0.061293  0.056617       3        1   4.585252
3      0.059896  0.038471       4        1   4.540818
5      0.056150  0.060493       5        1   3.948153
6      0.053171  0.062134       6        1   3.832435
8      0.054443  0.062105       7        1   3.739021
0      0.039852  0.061100       8        1   2.556456
1      0.028726  0.058304       9        1   2.510651
7      0.038814  0.060839      10        1   2.374401, topic_info=           Term          Freq         Total Category  logprob  loglift
35         size  12050.000000  12050.000000  Default  30.0000  30.0000
44          fit  11253.000000  11253.000000  Default  29.0000  29.0000
64         wear  10254.000000  10254.000000  Default  28.0000  28.0000
5          love  10354.000000  10354.000000  Default  27.0000  27.0000
42          top   9190.000000   9190.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
4994  defiantly      0.089994      1.628377  Topic10 -11.9226   0.8448
8615  crosswrap      0.089994      2.019398  Topic10 -11.9226   0.6296
8622   platinum      0.089994      1.099134  Topic10 -11.9226   1.2379
208        stat      0.089994      1.067591  Topic10 -11.9226   1.2670
567     keyhole      0.089994      1.064648  Topic10 -11.9226   1.2698

[589 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0        10  0.998195  absolutely
923       1  0.690581         adn
1624      1  0.990940   aesthetic
143       1  0.999451        also
252      10  0.997223       ankle
...     ...       ...         ...
241       1  0.999517         way
64        6  0.999870        wear
344       8  0.995323        week
1649      1  0.983432        wise
31        6  0.999621        work

[122 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 10, 4, 6, 7, 9, 1, 2, 8])

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -7.194378421192711

Coherence Score:  0.6378670459995781


# Justifications and questions

1. Topic 1 are about clothes, features of clothes, and behaviors and feelings of customers.  It is the most prevalent topic, much larger than any other topic, which is as expected because the data is about women's clothes reviews.
2. From the graph, we can see topic 1 and topic 2 both have no overlapping area with others and they are fairly seperated from the other, which means they are both good topics. But we look at the words in topic 2, most of them seem highly relative to topic 1, like 'look', 'sleeve' and 'short'. Topic 2 is hard to understand what this topic represents.
3. Topics expect topic 1 and topic 2 are almost overlapped with each other, clustering at the top-right corner. And it tells us that the number of topics we chose here is too large. We can choose other smaller number of topics to get a better coherence score.